### Dicts

#### Keys hashables
Las keys de un diccionario deben ser obligatoriamente hashables, es decir, el tipo de dato debe ser atómicamente inmutable (str, bytes, int, float).

In [19]:
try:
    print(hash('python'))  # hashable
    print(hash(3))  # hashable
    print(hash(3.6))  # hashable
    print(hash(['python']))  # unhashble
except Exception as e:
    print(f'Error: {e}')

-8616380704418249502
3
1383505805528216579
Error: unhashable type: 'list'


#### Diferentes formas de crear diccionarios

In [21]:
a = {'one': 1, 'two': 2, 'three': 3}
b = dict(one=1, two=2, three=3)
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('one', 1), ('two', 2), ('three', 3)])
e = dict({'one': 1, 'two': 2, 'three': 3})
a == b == c == d == e

True

#### dict comprehensions

In [27]:
SPAIN_ZIP_CODES = [
    ('46000', 'Valencia'),
    ('08000', 'Barcelona'),
    ('28000', 'Madrid'),
]
codes_by_city = {city: zip_code for zip_code, city in SPAIN_ZIP_CODES}
codes_by_city

{'Valencia': '46000', 'Barcelona': '08000', 'Madrid': '28000'}

#### setdefault()
Permite agregar valores sobre una key no existente al diccionario, y además devuelve dicho valor

In [32]:
product_prices = [
    ('product_1', 5),
    ('product_2', 10),
    ('product_1', 5),
    ('product_2', 1),
]
price_by_product = {}
for product, price in product_prices:
    price_by_product.setdefault(product, []).append(price)
price_by_product

{'product_1': [5, 5], 'product_2': [10, 1]}

#### collections.defaultdict

In [34]:
from collections import defaultdict
product_prices = [
    ('product_1', 5),
    ('product_2', 10),
    ('product_1', 5),
    ('product_2', 1),
]
price_by_product = defaultdict(list)
for product, price in product_prices:
    price_by_product[product].append(price)
price_by_product

defaultdict(list, {'product_1': [5, 5], 'product_2': [10, 1]})

### Variaciones de diccionarios

#### collections.OrderedDict
Mantiene las keys en el orden en el que han sido insertadas.

In [3]:
from collections import OrderedDict
my_ordered_dict = OrderedDict({
    'one': 1,
    'two': 2,
    'three': 3,
})
[key for key in my_ordered_dict.keys()]

['one', 'two', 'three']

#### collections.Counter
Incrementa un entero por cada coincidencia de una key dentro del diccionario.

In [14]:
def print_occurrences(my_counter):
    print(f'\n{my_counter}')
    for letter, occurrences in my_counter.items():
        print(f'Letter: {letter} -> {occurrences} occurrences')
    
from collections import Counter
my_counter = Counter('abecedario')
print_occurrences(my_counter)
my_counter.update('asdfasfasdfasdf')
print_occurrences(my_counter)

print('\nTop 3')
for letter, occurrences in my_counter.most_common(3):
    print(f'Letter: {letter} -> {occurrences} occurrences')


Counter({'a': 2, 'e': 2, 'b': 1, 'c': 1, 'd': 1, 'r': 1, 'i': 1, 'o': 1})
Letter: a -> 2 occurrences
Letter: b -> 1 occurrences
Letter: e -> 2 occurrences
Letter: c -> 1 occurrences
Letter: d -> 1 occurrences
Letter: r -> 1 occurrences
Letter: i -> 1 occurrences
Letter: o -> 1 occurrences

Counter({'a': 6, 'd': 4, 's': 4, 'f': 4, 'e': 2, 'b': 1, 'c': 1, 'r': 1, 'i': 1, 'o': 1})
Letter: a -> 6 occurrences
Letter: b -> 1 occurrences
Letter: e -> 2 occurrences
Letter: c -> 1 occurrences
Letter: d -> 4 occurrences
Letter: r -> 1 occurrences
Letter: i -> 1 occurrences
Letter: o -> 1 occurrences
Letter: s -> 4 occurrences
Letter: f -> 4 occurrences

Top 3
Letter: a -> 6 occurrences
Letter: d -> 4 occurrences
Letter: s -> 4 occurrences


#### collections.UserDict
Una implementación pura de Python para trabajar con mapeos.
Mientras las anteriores variacioens de diccionarios ya estaban listas para ser usadas, el UserDict está diseñada para crear nuevas clases a raíz de ella.

#### Método __missing__()
Permite manejar la excepción lanzada cuando no se encuentra una key dentro de un diccionario (dict[key]). El método no viene implementado en la clase genérica dict, con lo que habría que herederla para implementar dicha funcionalidad, aunque la mejor manera de implementar dicho método es la de heredar la clase collections.UserDict. 

In [15]:
# dict estándar
my_dict = {'1': 1, '2': 2}
print(my_dict)
print(my_dict['1'])
print(my_dict[2])  # raise a excepction because the key 2 not in the dict

{'1': 1, '2': 2}
1


KeyError: 2

In [18]:
class StrKeyDict(dict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
        
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()

my_str_dict = StrKeyDict({'1': 1, '2': 2})
print(my_str_dict)
print(my_str_dict['1'])
print(my_str_dict[2])  # not raise an error, because the method __missing__ realize the cast to str to get the value

{'1': 1, '2': 2}
1
2


In [2]:
import collections
class StrKeyDict0(collections.UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item
        
my_str_dict0 = StrKeyDict0({'1': 1, '2': 2})
my_str_dict0[3] = 3
print(my_str_dict0)
print(my_str_dict0['1'])
print(my_str_dict0[2])

{'1': 1, '2': 2, '3': 3}
1
2


#### types.MappingProxyType
En ciertos escenarios es útil disponer de un diccionario en el cuál no se puede realizar ninguna modificación, sino simplemente usarlo para leer la información obtenida en él.
Una instancia de MappngProxyType sirve como un reflejo de solo lectura el diccionario sobre el que se está instanciando, es decir, que cualquier modificación sobre el diccionario original se verá reflejado en la nueva instancia.

In [4]:
from types import MappingProxyType
my_dict = {'language': 'Python'}
my_dict_proxy = MappingProxyType(my_dict)
print(my_dict_proxy)
print(my_dict_proxy['language'])  # is possible get values
try:
    my_dict_proxy['version'] = 3  # but is not possible set new values
except:
    pass
my_dict['version'] = 3
print(my_dict_proxy)

{'language': 'Python'}
Python
{'language': 'Python', 'version': 3}
